<a href="https://colab.research.google.com/github/ndvp39/CloudComputing-tirgul/blob/main/Project/Chatbot/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install nltk
#!pip install requests beautifulsoup4

# activate only if run this file directly and not from searchEngine
'''
from bs4 import BeautifulSoup
from nltk.chat.util import Chat, reflections
from firebase_admin import credentials
from firebase_admin import firestore
import firebase_admin
import nltk
import requests

!git clone "https://github.com/ndvp39/CloudComputing-tirgul"
'''

%cd /content/CloudComputing-tirgul/Project/Index/
%run index.ipynb

# Download necessary NLTK data
# nltk.download('punkt')
# nltk.download('wordnet')

/content/CloudComputing-tirgul/Project/Index


In [ ]:
index = get_index_from_db()

In [ ]:
# Retrieves a link from a word in the index.
def getLinkFromString(w,linkID):
  for word, links in index.items():
    if(word == w):
      return list(links.keys())[linkID]

In [ ]:
# Retrieves the first paragraph from a given URL for a chatbot response.
def get_first_paragraph_from_link(url):
    try:
        # Make an HTTP request
        response = requests.get(url)
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.text, 'html.parser')
            # Find the first <p> tag with the specified ID
            h2_tag = soup.find('h2', {'id': 'overview'})
            if h2_tag:
              para = h2_tag.find_next('p')
              if para:
                return para.get_text()
              else:
                return
                #return "Paragraph after h2 with id: overview not found."
            else:
              div_tags = soup.find_all('div', {'class': 'rh-generic--component'})
              # Check if there is at least a second occurrence
              if len(div_tags) >= 2:
                second_div_tag = div_tags[1]  # Select the second occurrence
                para = second_div_tag.find_next('p').get_text()
              elif len(div_tags) == 1:
                first_div_tag = div_tags[0]  # Select the first occurrence
                para = first_div_tag.find_next('p').get_text()
              else:
                return "An error occurred. If this issue persists please re-try in a few seconds."
            return para
        else:
            return f"Failed to fetch the page. Status code: {response.status_code}"
    except Exception as e:
        return f"An error occurred: {e}"

In [ ]:
# Converts a list of patterns and their associated responses into a dictionary format for db use.
def patterns_to_dict(patterns):
    patterns_dict = []
    for pattern, responses in patterns:
        pattern_dict = {
            'pattern': pattern,
            'responses': responses
        }
        patterns_dict.append(pattern_dict)
    return patterns_dict

In [ ]:
# Creates patterns for conversational responses based on indexed words and their associated links.
def create_patterns():
  for word, links in index.items():
    answers = []
    for i in range(len(links)):
      link = getLinkFromString(word,i)
      # Get the first paragraph from the link
      first_para = get_first_paragraph_from_link(link)
      if first_para == "": # also need to remove dates
        continue
      answers.append(first_para + f"<br>you can read more in the link: {link}")
    # Create a pattern tuple
    to_append = ( r'(.*)' + word + r'(.*)', answers)
    patterns.append(to_append)

In [ ]:
# Upload the patterns to the db for faster use.
def upload_to_db(data_list, collection_name):
  try:
    for data in data_list:
        doc_ref = db.collection(collection_name).add(data)
    print("All documents uploaded successfully.")
  except Exception as e:
      print("Error:", e)

In [ ]:
# Get the patterns (responses) from db for quick answers from the chatbot.
def get_patterns_from_db(db_collection_name):
    patterns = []
    collection_ref = db.collection(db_collection_name)
    docs = collection_ref.get()
    for doc in docs:
        data = doc.to_dict()
        pattern = r'(.*)' + data['pattern'] + r'(.*)'
        responses = data['responses']
        patterns.append((pattern, responses))
    return patterns

In [ ]:
# Helper function to remove a specific response. ### CURRENTLY NOT IN USE ###
def delete_response_from_db(bot_response):
    collection_ref = db.collection("ChatBotPatterns")
    docs = collection_ref.get()
    for doc in docs:
        doc_data = doc.to_dict()
        # Check if the document has a 'responses' field
        if 'responses' in doc_data:
            # Iterate over responses in the 'responses' array
            updated_responses = []
            for response in doc_data['responses']:
                # Check if the response contains the search string
                if bot_response in response:
                    # If it does, remove the string from the array
                    continue
                else:
                    updated_responses.append(response)
            # Update the document with the modified 'responses' array
            doc.reference.update({'responses': updated_responses})

Main section for chatbot.ipynb

In [ ]:
patterns = [
    (r'hi|hello|hey', ['Hello!', 'Hi there!', 'Hey!']),
    (r'how are you?', ['I\'m good, thank you!', 'I\'m doing well, thanks for asking.']),
    (r'what is your name?', ['You can call me PantherBOT.', 'I go by the name PantherBOT.']),
    (r'your name (.*)', ['You can call me PantherBOT.', 'I go by the name PantherBOT.']),
    (r'give me good cloud website', ['Sure, here: www.redhat.com', 'ofCourse: you can go to amazon, which works with RedHat website check both']),
    (r'What kinds of clouds are there ?', ['There are three main kinds of clouds: private, public, and hybrid would you like me to explain further? ', 'public, hybrid, private - in secured order']),
    (r'(.*)kinds of clouds(.*)?(.*)', ['There are three main kinds of clouds: private, public, and hybrid would you like me to explain further? ', 'public, hybrid, private - in secured order']),
    (r'please explain further about clouds(.*)', ['cloud_explanation', "for the moment I too tired, may be later I will explain. You should've listen in class "])
    # You can add more patterns and responses here
]

db_collection_name = "ChatBotPatterns"
cred = credentials.Certificate("/content/CloudComputing-tirgul/Project/Json/sak.json")
if not firebase_admin._apps:
  firebase_admin.initialize_app(cred)
db = firestore.client()

##### The long functions -> do not activate unless you want to change bot dialog ######

# create_patterns()
final_patterns = patterns_to_dict(patterns)

###### Uploading to DB ######
# upload_to_db(final_patterns, db_collection_name)

# print what goes inside the db
# print(final_patterns)
# print(get_patterns_from_db(db_collection_name))
